# Decode an Alien Dictionary

There is a new alien language which uses the latin alphabet. However, the order among letters are unknown to you. You receive a list of non-empty words from the dictionary, where words are sorted lexicographically by the rules of this new language. Derive the order of letters in this language.

Example 1:

Given the following words in dictionary,

[
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]

The correct order is: "wertf".

Example 2:

Given the following words in dictionary,

[
  "z",
  "x"
]

The correct order is: "zx".

Example 3:

Given the following words in dictionary,

[
  "z",
  "x",
  "z"
]

The order is invalid, so return "".

Note:
- You may assume all letters are in lowercase.
- You may assume that if a is a prefix of b, then a must appear before b in the given dictionary.
- If the order is invalid, return an empty string.
- There may be multiple valid order of letters, return any one of them is fine.

## Solution

Since these words are lexigraphically ordered,  we know that for any two words a and b, going from the left to the right, for the first letter that differs in them, letter_a < letter_b

e.g.

aaaaac

aaaaad

c < d

berted
cdfeed

b < c, nothing else can be assumed

So, given a pair of words, in a known order, the only thing that can be learned is the ordering of two letters.

We've now defined the most basic unit of computation in a dynamic programming solution.  We create a function  "process_pair(a, b, library)", and learn which letters are behind and in front of another given letter.

We pass through the words in each pair (word_i, word_i+1).  As we do this, we collect a map of relationships, for 

letter_k -> set_of_letters_less_than_letter_k and set_of_letters_greater_than_letter_k

After this pass, we've learned everything there is be be learned from the ordering.

With this map of relationships, we can generate our ordering by iterating through the map, and an output array, then inserting letter after we've passed all the letters foudn the be less than the current letter.

In [29]:
def alienOrder(words):
    """
    :type words: List[str]
    :rtype: str
    """
    # this is the dynamic programming part, our sub-problem is comparing pairs of words, 
    # finding where they differ first, and at those letters a < b
    
    # O(l), where l is the length of the longest word in the set of words
    def process_pair(a, b, lt_map):
        a_idx = 0
        b_idx = 0
        while a_idx < len(a) and b_idx < len(b):
            a_l = a[a_idx]
            b_l = b[b_idx]
            if not a_l in lt_map:
                lt_map[a_l] = set()
            if not b_l in lt_map:
                lt_map[b_l] = set()
            if a_l == b_l:
                a_idx += 1
                b_idx += 1
                continue
            else:
                if b_l in lt_map[a_l]:
                    lt_map = ""
                    break
                else:
                    lt_map[b_l].add(a_l)
                    break

        return lt_map
    
    if len(words) == 0:
        return None
    if len(words) == 1:
        out = ""
        processed = set()
        for letter in words[0]:
            if not letter in processed:
                out += letter
                processed.add(letter)
            elif not out[-1] == letter:
                return ""
            else:
                continue
        return out
            
    
    lt_map = {}
    aw_idx = 0
    bw_idx = 1
    # O((n-1)*l), where N is the number of words and L is the length of the longest word
    while bw_idx < len(words):
        a = words[aw_idx]
        b = words[bw_idx]
        lt_map = process_pair(a, b, lt_map)
        if lt_map == "":
            return ""
        aw_idx += 1
        bw_idx += 1

    # generate ordering with learned relationship    
    
    out = []
    # O(c**2), where c is the number of unique characters, a max of 26 in this case
    # precicely it's O((n+1)*n/2), where n is a max of 26, or 351 operations
    for k, v in lt_map.items():
        idx = 0
        while len(v) > 0 and idx < len(out):
            l = out[idx]
            if l in v:
                v.remove(l)
            idx += 1
        out = out[:idx] + [k] + out[idx:]
    
    return out

In [30]:
words = [
  "wrt",
  "wrf",
  "er",
  "ett",
  "rftt"
]

print(alienOrder(words))
print(alienOrder([ "z", "x" ]))
print(alienOrder([ "z", "x", "z" ]))

['w', 'e', 'r', 't', 'f']
['z', 'x']

